In [6]:
!pip install pandas
!pip install openpyxl
!pip install requests
!pip install beautifulsoup4
!pip install newspaper3k
!pip install selenium
!pip install openai
!pip install serpapi
!pip install googlesearch-python
!pip install google-search-results
!pip install GoogleNews
!pip install "httpx[http2]"
!pip install "lxml[html_clean]"

In [7]:
from tabulate import tabulate
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import time
import random
import re
from datetime import datetime
from GoogleNews import GoogleNews
from serpapi import GoogleSearch
import pandas as pd
from urllib.parse import quote_plus
from serpapi.google_search import GoogleSearch

## Data frame unificado


In [8]:
api_key = "3e815855bc007210de64f13b41d3a9dd56da0540c5da030c9b4baf6142e0c830"


In [ ]:
api_key = "3e815855bc007210de64f13b41d3a9dd56da0540c5da030c9b4baf6142e0c830"

# ===================================================
# 🔹 FUNCIONES DE SCRAPING
# ===================================================

def link_valido(url):
    try:
        r = requests.head(url, timeout=5, allow_redirects=True)
        return r.status_code == 200
    except:
        return False

def buscar_noticias_generales(nombre_fuente, url_base, termino_busqueda, max_resultados=15):
    """Scrapea noticias desde sitios como El Tiempo, ADE o FECODE."""
    print(f"\n🔍 Buscando en {nombre_fuente}: {termino_busqueda}")
    datos = []
    termino_codificado = quote_plus(termino_busqueda)
    url_busqueda = f"{url_base}{termino_codificado}"

    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        response = requests.get(url_busqueda, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        articulos = soup.find_all('article', limit=max_resultados)
        print(f"📰 Encontrados {len(articulos)} artículos en {nombre_fuente}")

        for art in articulos:
            titulo = (art.find('h2') or art.find('h3'))
            enlace = art.find('a', href=True)

            if not titulo or not enlace:
                continue

            link = enlace['href']
            if not link.startswith('http'):
                link = f"{url_base.rstrip('/')}/{link.lstrip('/')}"

            if not link_valido(link):
                continue

            descripcion = art.find('p').get_text(strip=True) if art.find('p') else ''
            fecha_elem = art.find('time')
            fecha = fecha_elem.get_text(strip=True) if fecha_elem else 'No especificada'

            datos.append({
                'fuente': nombre_fuente,
                'titulo': titulo.get_text(strip=True),
                'descripcion': descripcion,
                'fecha_publicacion': fecha,
                'url': link,
                'periodo_busqueda': termino_busqueda,
                'fecha_extraccion': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })

    except Exception as e:
        print(f"❌ Error en {nombre_fuente}: {e}")

    time.sleep(random.uniform(1.5, 3.5))
    return datos


def buscar_con_googlenews():
    """Extrae resultados desde GoogleNews API."""
    googlenews = GoogleNews(lang='es', encode='utf-8')
    googlenews.set_time_range('01/01/2023', '12/31/2023')
    resultados = []
    queries = [
            'paro docentes Bogotá',
            'paro maestros Colombia',
            'huelga de profesores Bogotá',
            'protesta Fecode 2023',
            'ADE Bogotá educación',
            # Variaciones con sinónimos y contexto
            'cese de actividades docentes',
            'suspensión de clases Bogotá',
            'manifestación de maestros Colombia',
            'docentes sin clases 2023',
            'reclamos del magisterio colombiano',
            'paro educativo nacional 2023',
            # Variaciones temporales
            'ministerio de educación paro 2023',
            'educadores en huelga 2023',
            'protesta del sindicato de maestros 2023',
            'movilización docente Bogotá 2023'
    ]
    for q in queries:
        googlenews.search(q)
        time.sleep(random.uniform(2, 4))
        res = googlenews.result()

        for r in res:
            resultados.append({
             'fuente': r.get('media', 'GoogleNews'),
             'titulo': r.get('title'),
             'descripcion': r.get('desc', ''),
             'url': r.get('link', ''),
             'fecha_publicacion': r.get('date', ''),
             'periodo_busqueda': q,
             'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })
    print(f"✅ GoogleNews: {len(resultados)} resultados")
    return resultados


def buscar_con_serpapi(api_key):
    """Busca en Google usando SerpAPI y devuelve resultados compatibles."""
    print("\n🔍 Buscando con SerpAPI...")
    resultados = []
    queries = [
        '("suspensión de clases" OR "cese de actividades" OR "interrupción académica") Bogotá 2023',
        '("paro de maestros" OR "paro docente" OR "huelga de profesores" OR "protesta de docentes") Bogotá 2023',
        '("interrupción de clases" OR "cancelación de clases" OR "no hay clases") Bogotá 2023',
        '("sindicato de maestros" OR "FECODE" OR "ADE Bogotá") AND (paro OR huelga OR protesta) 2023',
    ]

    for query in queries:
        print(f"  • {query}")
        params = {
            "engine": "google",
            "q": f"{query} after:2023-01-01 before:2023-12-31",
            "api_key": api_key,
            "num": 100,
        }
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            items = results.get("organic_results", [])
            for item in items:
                resultados.append({
                    'fuente': 'Google (SerpAPI)',
                    'titulo': item.get('title'),
                    'fecha_publicacion': '',
                    'descripcion': item.get('snippet', ''),
                    'url': item.get('link'),
                    'periodo_busqueda': query,
                    'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
        except Exception as e:
            print(f"❌ Error con SerpAPI en query '{query}': {e}")
            time.sleep(random.uniform(1, 2))
    print(f"✅ SerpAPI: {len(resultados)} resultados")
    return resultados
# ===================================================
# 🔹 EJECUCIÓN PRINCIPAL
# ===================================================
if __name__ == "__main__":
    SerpAPI_tkn1 = api_key

    resultados_totales = []
    print("\n Iniciando recolección de noticias del año 2023...")

    fuentes = {
        "El Tiempo": "https://www.eltiempo.com/buscar/",
        "El Espectador": "https://www.elespectador.com/buscador/",
        "Semana": "https://www.semana.com/buscador/?query=",
        "RCN Radio": "https://www.rcnradio.com/buscar?query=",
        "Noticias Uno": "https://canal1.com.co/buscar/?q=",
        "FECODE": "https://fecode.edu.co/?s=",
        "ADE Bogotá": "https://adebogota.org/index.php/buscar?q=",
        "Ministerio de Educación": "https://www.mineducacion.gov.co/portal/buscador?search_api_fulltext=",
        "Portafolio": "https://www.portafolio.co/buscar?q="
    }

    terminos_busqueda = [
        "paro docentes ",
        "paro maestros ",
        "huelga de profesores ",
        "protesta docente 2023",
        "cese de actividades escolares 2023",
        "manifestación de educadores 2023",
        "Fecode paro ",
        "ADE Bogotá 2023",
        "sindicato de maestros 2023",
        "magisterio colombiano 2023",
        "educadores ADE Fecode 2023",
        "Ministerio de Educación paro docente 2023",
        "reforma educativa 2023 Colombia",
        "clases suspendidas Bogotá ",
        "negociaciones Fecode Gobierno ",
        "condiciones laborales docentes 2023"
    ]

    for fuente, url_base in fuentes.items():
        for termino in terminos_busqueda:
            resultados_totales.extend(
                buscar_noticias_generales(fuente, url_base, termino)
            )
            time.sleep(random.uniform(2, 4))

    # --- GoogleNews ---
    resultados_totales.extend(buscar_con_googlenews())

    # --- SerpAPI ---
    resultados_totales.extend(buscar_con_serpapi(SerpAPI_tkn1))

    # ===================================================
    # 🔹 Unir todo y guardar CSV
    # ===================================================
    df = pd.DataFrame(resultados_totales)
    df = df[df["url"].str.contains("http", na=False)]
    df = df.drop_duplicates(subset='url', keep='first')
df.to_csv("noticias_paros_docentes_2023.csv", index=False, encoding="utf-8-sig")

print(f"\n Total de noticias guardadas: {len(df)}")
print(" Archivo: noticias_paros_docentes_2023.csv")
print("\n PROCESO COMPLETADO CON ÉXITO ")



 Iniciando recolección de noticias del año 2023...

🔍 Buscando en El Tiempo: paro docentes 
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: paro maestros 
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: huelga de profesores 
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: protesta docente 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: cese de actividades escolares 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: manifestación de educadores 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: Fecode paro 
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: ADE Bogotá 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: sindicato de maestros 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: magisterio colombiano 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: educadores ADE Fecode 2023
📰 Encontra